# Python DB API

# PyODBC

## SQL Server Connection

In [1]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pyodbc

In [10]:
def create_server_connection(driver, server, database, user, password):
    conn = None
    try:
        conn = pyodbc.connect(DRIVER = driver, SERVER = server, DATABASE = database, UID = user, PWD = password)
        print("Successful")
    except Error as err:
        print(f"Error: '{err}'")
    return conn

In [3]:
## kullanıcı adı ile bağlanma
driver = '{ODBC Driver 17 for SQL Server}'
server = 'localhost' 
database = 'master' 
user = 'sa' 
password = 'maltepe35'
conn = create_server_connection(driver, server, database, user, password)

Successful


In [4]:
pyodbc.drivers()

['SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 13 for SQL Server',
 'ODBC Driver 17 for SQL Server',
 'PostgreSQL ANSI(x64)',
 'PostgreSQL Unicode(x64)']

In [9]:
#conn.close()

In [11]:
#windows aut bağlanma
conn_string = "driver={ODBC Driver 17 for SQL Server}; server=localhost; database=master; TRUSTED_CONNECTION=yes;"
conn = pyodbc.connect (conn_string)
crs = conn.cursor()
conn.autocommit=True
create_db_syntax = 'CREATE DATABASE SAMPLEDB_1'
def create_database (conn, create_db_syntax):
    conn = conn.cursor()
    try:
        conn.execute(create_db_syntax)
        print("Database is Created Succesfully")
    except Exception as err:
        print(f"Error:'{err}'")
create_database(conn, create_db_syntax)

Database is Created Succesfully


In [15]:
def execute_query(conn, query):
    conn = conn.cursor()
    try:
        conn.execute(query)
        conn.commit()
        print("Query successful")
    
    except Exception as err:
        print(f"Error: '{err}'")

In [16]:
query = 'USE SAMPLEDB_1'

In [17]:
execute_query(conn, query)

Query successful


In [18]:
query = 'CREATE TABLE TestA ( \
             ID int IDENTITY(1,1) NOT NULL, \
             FirstName varchar(255) NOT NULL, \
             LastName varchar(255) NOT NULL, \
             PRIMARY KEY(ID))'
execute_query(conn, query)

Query successful


In [20]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES ('Bob', 'Marley')")

In [21]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Bruce', 'Lee')

In [22]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Tom', 'Hanks')
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Bart', 'Simpson')

In [23]:
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Tom', 'Cat')
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Jerry', 'Mouse')
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Owen', 'William')
crs.execute("INSERT TestA (FirstName, LastName) VALUES (?,?)",'Stefan', 'Müller')

# Delete

In [25]:
crs.execute("DELETE from TestA where FirstName = ?", 'Jerry')

In [26]:
crs.execute("DELETE from TestA where FirstName = ?", 'Stefan')

# RowCount

In [27]:
crs.rowcount

1

In [28]:
print(crs.rowcount,"row(s) deleted")

1 row(s) deleted


# Reading Data

# Fetchone

In [29]:
crs.execute('SELECT FirstName, LastName FROM TestA')
row = crs.fetchone() #tablodaki ilk kaydı getirir
row

('Bob', 'Marley')

In [30]:
row = crs.fetchone() #tablodaki ilk kaydı getirir
row

('Bruce', 'Lee')

In [31]:
row = crs.fetchone() #tablodaki ilk kaydı getirir
row

('Tom', 'Hanks')

In [32]:
row.FirstName

'Tom'

In [33]:
crs.execute('SELECT ID, FirstName, LastName FROM TestA')
rows = crs.fetchall()
rows

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Tom', 'Cat'),
 (7, 'Owen', 'William')]

In [34]:
for row in rows:
    print(row.ID, row.FirstName, row.LastName)

1 Bob Marley
2 Bruce Lee
3 Tom Hanks
4 Bart Simpson
5 Tom Cat
7 Owen William


In [35]:
crs.execute("""SELECT ID, FirstName, LastName
                FROM TestA""")
rows = crs.fetchall()
rows

[(1, 'Bob', 'Marley'),
 (2, 'Bruce', 'Lee'),
 (3, 'Tom', 'Hanks'),
 (4, 'Bart', 'Simpson'),
 (5, 'Tom', 'Cat'),
 (7, 'Owen', 'William')]

In [36]:
for row in rows:
    print(row.ID, row.FirstName, row.LastName)

1 Bob Marley
2 Bruce Lee
3 Tom Hanks
4 Bart Simpson
5 Tom Cat
7 Owen William


In [37]:
crs.execute("""SELECT ID, FirstName, LastName
               FROM TestA
               WHERE FirstName = ? and LastName = ?""", ['Owen', 'William'])
rows = crs.fetchall()
rows

[(7, 'Owen', 'William')]

# Fetchmany

In [38]:
crs.execute("""SELECT ID, FirstName, LastName
               FROM TestA""")
rows = crs.fetchmany(2)
rows

[(1, 'Bob', 'Marley'), (2, 'Bruce', 'Lee')]

# FetchVal

In [63]:
crs.execute("SELECT MAX(ID) FROM TestA").fetchval()

7

In [64]:
crs.execute("SELECT FirstName FROM TestA").fetchval()


'Bob'

In [65]:
crs.execute("SELECT FirstName, LastName FROM TestA").fetchval()

'Bob'

# Pandas Methods

In [66]:
import pandas as pd

In [67]:
query = """SELECT ID, FirstName, LastName
               FROM TestA"""

In [72]:
def read_query (conn, query): #connection nesnesi yukarıda tanımlanmıştı.
    crs = conn. cursor()
    results = None
    crs.execute(query)
    results = crs.fetchall()
    return results

In [71]:
read_query(conn, query)

Error: ('HY000', '[HY000] [Microsoft][ODBC Driver 17 for SQL Server]Connection is busy with results for another command (0) (SQLExecDirectW)')

In [73]:
query = 'USE SampleSales'
execute_query(conn,query)

Error: '('HY000', '[HY000] [Microsoft][ODBC Driver 17 for SQL Server]Connection is busy with results for another command (0) (SQLExecDirectW)')'
